# PayIn and Escrow Identification (left to right)
This notebook queries for PayIn transactions matching reviews to vendors that were active on both Berlusconi and Cryptonia Market and have already been identified in previous work. The results were not used during the rest of the project.

In [1]:
import pandas as pd
import mysql.connector
from decimal import Decimal
from neo4j import GraphDatabase
from datetime import timedelta

In [2]:
#Load username and password
df_pw = pd.read_csv("pw.csv", delimiter=",")

In [3]:
#Establish connection to SQL db
df_pw = pd.read_csv("pw.csv", delimiter=",")
con = mysql.connector.connect(
    host="localhost",
    user=df_pw["name"][0], 
    passwd=df_pw["password"][0],
    database="DarknetScrape")
cursor = con.cursor()

In [4]:
#Establish connection to Neo4j
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))
session = db.session()

In [5]:
#Load known Cryptonia Market vendor addresses from Jochen
j_addresses=pd.read_csv('candidates_final.csv')
j_addresses=j_addresses.drop(columns=['Unnamed: 0'])
j_addresses

,Vendor,Address
0,CNSHarmony,34Xo7fKRe9MjYBf3tyJjAFg9qX2kuFuhA1
1,TheHoneymonster,1Ew7BnRn3Q3xkpKMpgc7r5YFrkQXpwXMz7
2,EUCarder,1Nou6Q9qmxbi48m6eSeYy7KHvSSiWFMq5y
3,kaliforniaProducts,12oCEud6XZm9ePxiUnbiUvsjUwEUs8GsgE
4,DerFeinschmecker,bc1q63eumgx8a0esl4uu2zdusheadf79nxg5zup8h0
...,...,...
194,GrandWizardsLair,1LoXrv7WQ5H6m2BdY99VpYsSdKcGyxwke4
195,clockwerk,14VSzUV8NffH3miGAGzdAx62gVumgw3afr
196,hashking,19bBP43Mh4edueHie5akFBiqRngjUcexeK
197,ChipChipperson,1LaxMbZ2zh6H7PuKFy5v9axCk1EPBpe8r4


In [6]:
#Load all Berlusconi Market reviews
berlusconi_reviews  = pd.read_csv('reviews_price_btc.csv')
berlusconi_reviews.time = pd.to_datetime(berlusconi_reviews.time)

In [7]:
#Query for vendors active on Cryptonia Market from SQL db
query = """select a.vendor_id, b.username from (
SELECT vendor_id, count(*) as count
from vendors
group by vendor_id
) as a
JOIN (SELECT vendor_id, username from vendors where market = 'Cryptonia') b
on a.vendor_id = b.vendor_id
where a.count = 2;
"""
cursor.execute(query)
df = cursor.fetchall()

#Filter for vendors that Jochen already identified
df=df[df['username'].isin(j_addresses['Vendor'])]
#Filter BM reviews for reviews of vendors that Jochen identified
berlusconi_reviews = berlusconi_reviews[berlusconi_reviews["vendor_id"].isin(df["vendor_id"])]
berlusconi_reviews

,Unnamed: 0,price_eur,time,customer,vendor_id,vendor_review_id,price_btc_review,price_btc_reviews_satoshi
26608,26608,165.14,2017-10-09 14:35:07,tk [5],a081033ab42c85ae4101549cd64ff4cb,105344,0.040876,4.087624e+06
26607,26607,401.17,2017-10-19 19:25:48,tk [5],a081033ab42c85ae4101549cd64ff4cb,105343,0.083459,8.345885e+06
25551,25551,277.69,2017-10-20 13:31:27,I***5 [2],8438ba1a66f83580db67292ecced8f9c,104261,0.056041,5.604125e+06
25550,25550,104.36,2017-10-23 22:06:34,np [3],8438ba1a66f83580db67292ecced8f9c,104260,0.020842,2.084157e+06
26606,26606,110.91,2017-12-05 23:06:20,a**c [1],a081033ab42c85ae4101549cd64ff4cb,105342,0.011203,1.120269e+06
...,...,...,...,...,...,...,...,...
21974,21974,38.88,2019-07-29 08:43:06,c*4| [19],135137a5e88e1fa749c1374f51048773,100611,0.004563,4.562577e+05
22234,22234,20.73,2019-07-29 09:30:40,c** [17],1db74355bd33521d281b3e15cc6dbe21,100874,0.002433,2.432670e+05
24868,24868,18.23,2019-07-29 09:59:25,m**F [9],6233b71b602efb80ef560af4480af1fa,103560,0.002139,2.139295e+05
24020,24020,86.34,2019-07-29 10:01:51,w***g [15],5c6b492caa2e2af07032efbaa9a9dd46,102688,0.010132,1.013202e+06


In [8]:
#Query for payin TAs with change and first level BM escrow addresses & first level mixing TA 
#based on inSum, date and outDegree taken from reviews 
escrow_addresses_v1 = pd.DataFrame(columns=['inDegree', 'outDegree', 'address','vendor'])
for i in berlusconi_reviews.itertuples():
    neo_results = []
    neo_results2 = []
    start = str(i.time.date()- timedelta(days=3))
    end = str(i.time.date())
    minSum = str(round(i.price_btc_reviews_satoshi,0))
    maxSum = str(round(i.price_btc_reviews_satoshi*1.4,0))
    neo_query = session.run("""
    WITH '""" + start + """' as startDate,  '""" + end + """' as endDate, """
    + minSum + """ as minSum, """ + maxSum +""" as maxSum
    MATCH (payin:Transaction) 
        WHERE  t.inDegree = 1 and ( t.outDegree = 2 OR t.outDegree = 1 ) AND date(startDate) <= t.date <= date(endDate) 
        and minSum <= t.inSum<=maxSum
    WITH t, minSum, startDate, endDate
    MATCH (payin)-[r:RECEIVES]->(escrow:Address),
        (payin)-[r1:RECEIVES]->(change:Address)-[s:SENDS]->(payin)
        WHERE minSum  <= r.value <= minSum * 1.05 AND c <> a AND 2000000 < escrow.totalMoneyReceived < 200000000 
        AND escrow.minDate>date('2017-09-08') AND escrow.maxDate<date('2019-07-31')
    WITH a, r, startDate, endDate
    MATCH (a)-[s:SENDS]->(mix:Transaction)
        WHERE date(startDate) <= mix.date <= date(endDate) AND r.value >= s.value 
    RETURN a, r.value
    """)
    for r in neo_query:
        neo_results2.append(r[0])
        neo_results.append(r[1])

    
    df_neo4j2 = pd.DataFrame(neo_results2)
    df_neo4j2["vendor"] = i.vendor_id
    df_neo4j2["review"] = i.vendor_review_id
    df_neo4j2["rvalue"] = pd.Series(neo_results)
    escrow_addresses_v1 = pd.concat([escrow_addresses_v1,df_neo4j2])

escrow_addresses_v1.to_csv("QueryResultsTransaction1EscrowActiveOnBoth0506.csv")    
display(escrow_addresses_v1.head())

<ipython-input-8-8876abb4a2f7>:38: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_neo4j2["rvalue"] = pd.Series(neo_results)


,inDegree,outDegree,address,vendor,maxDate,minDate,totalMoneyReceived,review,rvalue
0,2,2,18Hw3c4WcfRW6cdtfSP8MDsaNHJbN8bpKa,a081033ab42c85ae4101549cd64ff4cb,2017-11-17,2017-10-06,4410000.0,105344.0,4180000.0
1,2,2,1BEVjYniwpY4itFtYLhautNpSJqDtPuYre,a081033ab42c85ae4101549cd64ff4cb,2017-10-06,2017-09-27,4571961.0,105344.0,4096761.0
2,1,1,15u1M5iN4HAivHSjbgd37RjapYjZwbALG7,a081033ab42c85ae4101549cd64ff4cb,2017-10-06,2017-10-06,4254815.0,105344.0,4254815.0
3,22,22,1MR4x3Q4AGUs2PSaXD3HTwV6wpJSky5sgr,a081033ab42c85ae4101549cd64ff4cb,2017-12-27,2017-10-01,64373825.0,105344.0,4218000.0
4,22,22,1MR4x3Q4AGUs2PSaXD3HTwV6wpJSky5sgr,a081033ab42c85ae4101549cd64ff4cb,2017-12-27,2017-10-01,64373825.0,105344.0,4218000.0


In [8]:
escrow_addresses_v1=pd.read_csv("QueryResultsTransaction1EscrowActiveOnBoth0506.csv")

In [10]:
len(escrow_addresses_v1['address'].unique())

16553

In [11]:
#Why inDegree >20 ???
v2=escrow_addresses_v1.groupby(by='address').count().sort_values(by=['inDegree'], ascending=False)
v2=v2[v2['inDegree']>20]
v2=v2.reset_index()
escrow_addresses_v2=v2['address'].tolist()

In [12]:
escrow_addresses_v2_withUsername=escrow_addresses_v1[escrow_addresses_v1['address'].isin(escrow_addresses_v2)]
escrow_addresses_v2_withUsername=escrow_addresses_v2_withUsername[['address', 'vendor']]
escrow_addresses_v2_withUsername=escrow_addresses_v2_withUsername.merge(df, how="inner", left_on='vendor', right_on='vendor_id')
escrow_addresses_v2_withUsername=escrow_addresses_v2_withUsername.drop_duplicates()
escrow_addresses_v2_withUsername

,address,vendor,vendor_id,username
0,19jdijP7aCjZHP6644Y14fymzN6QQpZo7N,8438ba1a66f83580db67292ecced8f9c,8438ba1a66f83580db67292ecced8f9c,MrAddyDeals
30,1NFkRsmKxtK8T6o1GmNFaoh9ab6PyQfGLM,8438ba1a66f83580db67292ecced8f9c,8438ba1a66f83580db67292ecced8f9c,MrAddyDeals
37,33p5in7CipZZzeouNXinMfEnGdk13wSd8K,8438ba1a66f83580db67292ecced8f9c,8438ba1a66f83580db67292ecced8f9c,MrAddyDeals
75,3B3ajdw1Cdpo1cEsnEybQmnPhFrSPGmyyx,8438ba1a66f83580db67292ecced8f9c,8438ba1a66f83580db67292ecced8f9c,MrAddyDeals
108,3MKQZqCx7pGEzZ1aBPFTjAqHk4r8xNbWW6,8438ba1a66f83580db67292ecced8f9c,8438ba1a66f83580db67292ecced8f9c,MrAddyDeals
...,...,...,...,...
30407,3CCL5kes8ew8BiaZp5fQ2Do38G3FTTe6Yy,4f1c61d2184c48dca66028681c2ee183,4f1c61d2184c48dca66028681c2ee183,DrParagon
30437,3CRvV5WmN3MqzyEqX9daBU1ja2E6fit9nw,4f1c61d2184c48dca66028681c2ee183,4f1c61d2184c48dca66028681c2ee183,DrParagon
30444,1NZ1SSSGv5b6KVcJpfwhN5jJcQ9K5Hwmbe,4f1c61d2184c48dca66028681c2ee183,4f1c61d2184c48dca66028681c2ee183,DrParagon
30450,1P1aQQ12ZSQiFJFKhXoZxDY6mXKisLRhX2,4f1c61d2184c48dca66028681c2ee183,4f1c61d2184c48dca66028681c2ee183,DrParagon


In [13]:
#Query for connection in Neo4j between identified 1st level escrow addresses and Jochen's vendor adresses
#underlying assumption: Only works if the same coin that are spent by the customer are paid out to the vendor
# in other words: If a direct connection between customer and vendor exists in Neo4j
escrow_addresses_v3= pd.DataFrame(columns=['escrow', 'vendor', 'relationships'])
for row in escrow_addresses_v2_withUsername.itertuples():
    vendor=j_addresses[j_addresses['Vendor']==row.username]['Address'].iloc[0]
  
    neo_query = session.run("""
    MATCH p=(escrow:Address {address: '"""+row.address+"""' })-[:RECEIVES*1..5]-(vendor:Address {address: '"""+ vendor+"""'})
    RETURN collect(escrow) as escrow, collect(vendor) as vendor, collect(last(relationships(p))) as relationships""")
    for r in neo_query:
        escrow=r[0]
        vendor=r[1]
        relationships=r[2]
        escrow_addresses_v3.append({'escrow': escrow, 'vendor':vendor, 'relationships':relationships}, ignore_index=True)

display(escrow_addresses_v3.head())
escrow_addresses_v3.to_csv('escrow_addresses_v3.csv')

No data
Failed to read from defunct connection IPv4Address(('127.0.0.1', 7687)) (IPv4Address(('127.0.0.1', 7687)))


SessionExpired: Failed to read from defunct connection IPv4Address(('127.0.0.1', 7687)) (IPv4Address(('127.0.0.1', 7687)))